<div align='center'>

# OVERVIEW
</div>

This notebook aims to clarify **whether or not any of the suggested Identity & Verification services are acceptable for use for our Customer Support Center system**, as defined in the [`customer_support_rep_persona_01.md` persona document](../../customer_rep_persona_store/customer_support_rep_persona_01.md).

The suggested services are listed below:

1. **Keycloak** (OpenID Connect / OAuth2 Identity Provider)
2. **MinIO** (S3-compatible Object Storage for Document Uploads)

The criteria for qualifying a suggestion as a tool is defined below:

1. Python-based.
2. Open-source (free, avoid freemium as much as possible) customer-facing **and** producer-facing access.
3. Rate limit restrictions allow for appropriate edge-case testing.
4. Supports core architectural choice for LangChain ecosystem (e.g. supports asynchronous backends, etc).
5. I can currently access the tool as a human (i.e. confirmation that it is operable as a human, and has not been deprecated/restricted discreetly).
6. I can perform its expected basic operations.

## Pre-Notebook Initialization

1. Install Keycloak via Docker: `docker run -p 8080:8080 -e KEYCLOAK_ADMIN=admin -e KEYCLOAK_ADMIN_PASSWORD=admin quay.io/keycloak/keycloak:latest start-dev`
2. Install MinIO via Docker: `docker run -p 9000:9000 -p 9001:9001 minio/minio server /data --console-address ":9001"`
3. Create a Keycloak realm and client for testing.
4. Configure MinIO bucket for document storage.

<u>Notes</u>:
- Keycloak provides OAuth2/OpenID Connect authentication.
- MinIO provides S3-compatible object storage for document uploads.

# Required Modules


In [ ]:
import os
import requests
from dotenv import load_dotenv
load_dotenv()
from minio import Minio
from minio.error import S3Error
import io


## Keycloak


<ol>
<li> Python-based: <b>Available</b> (via REST API and python-keycloak library)
<li> Open-source: <b>Free</b> (Apache 2.0 License)

<li> Maximum Rate Limit Restriction:
<ul>
<li> Self-hosted: <b>No rate limits</b> (depends on infrastructure).
<li> Configurable brute-force protection and request throttling.
</ul>

<li> <b>Supports Core Architectural Choice</b> (REST API compatible with async HTTP clients)
<li> Service is confirmed to <b>still be active & accessible</b>
</ol>

### Expected Basic Operations

1. [Get Access Token (Client Credentials Grant)](https://www.keycloak.org/docs/latest/securing_apps/#_client_credentials_grant).
2. [Introspect Token](https://www.keycloak.org/docs/latest/securing_apps/#_token_introspection_endpoint).
3. [Get User Info](https://www.keycloak.org/docs/latest/securing_apps/#_userinfo_endpoint).
4. [Create User (Admin API)](https://www.keycloak.org/docs-api/latest/rest-api/index.html#_users).
5. [Validate JWT Token](https://www.keycloak.org/docs/latest/securing_apps/#_token_validation).
6. [Logout / Revoke Token](https://www.keycloak.org/docs/latest/securing_apps/#_logout).

**<u>Notes:</u>**
1. Keycloak provides both OpenID Connect and SAML 2.0 protocols.
2. The Admin REST API allows full programmatic control over realms, users, roles, and clients.
3. Keycloak supports social login integration (Google, GitHub, etc.).


### Setup

Keycloak provides a REST API for authentication operations. We'll create a simple client to interact with common identity operations.


In [ ]:
# Class Definition for Keycloak API calls

class Simple_Keycloak_Client():
    
    def __init__(
        self,
        server_url: str = None,
        realm: str = "master",
        client_id: str = None,
        client_secret: str = None
    ):
        self.server_url = server_url or os.getenv("KEYCLOAK_SERVER_URL", "http://localhost:8080")
        self.realm = realm
        self.client_id = client_id or os.getenv("KEYCLOAK_CLIENT_ID")
        self.client_secret = client_secret or os.getenv("KEYCLOAK_CLIENT_SECRET")
        self.token_endpoint = f"{self.server_url}/realms/{self.realm}/protocol/openid-connect/token"
        self.userinfo_endpoint = f"{self.server_url}/realms/{self.realm}/protocol/openid-connect/userinfo"
        self.introspect_endpoint = f"{self.server_url}/realms/{self.realm}/protocol/openid-connect/token/introspect"
        self.admin_url = f"{self.server_url}/admin/realms/{self.realm}"
        self.SUCCESS_MSG = "[INFO]: Request performed successfully!"
        self.ERROR_MSG = "[WARNING]:"
        self.access_token = None
        
    def get_access_token(self):
        """Get access token using client credentials grant"""
        try:
            response = requests.post(
                self.token_endpoint,
                data={
                    "grant_type": "client_credentials",
                    "client_id": self.client_id,
                    "client_secret": self.client_secret
                }
            )
            response.raise_for_status()
            self.access_token = response.json().get("access_token")
            print(self.SUCCESS_MSG)
            return response.json()
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None
    
    def introspect_token(self, token: str):
        """Introspect a token to validate and get metadata"""
        try:
            response = requests.post(
                self.introspect_endpoint,
                data={
                    "token": token,
                    "client_id": self.client_id,
                    "client_secret": self.client_secret
                }
            )
            response.raise_for_status()
            print(self.SUCCESS_MSG)
            return response.json()
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None
    
    def get_user_info(self, access_token: str = None):
        """Get user information from the userinfo endpoint"""
        token = access_token or self.access_token
        try:
            response = requests.get(
                self.userinfo_endpoint,
                headers={"Authorization": f"Bearer {token}"}
            )
            response.raise_for_status()
            print(self.SUCCESS_MSG)
            return response.json()
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None
    
    def get_well_known_config(self):
        """Get OpenID Connect discovery document"""
        try:
            url = f"{self.server_url}/realms/{self.realm}/.well-known/openid-configuration"
            response = requests.get(url)
            response.raise_for_status()
            print(self.SUCCESS_MSG)
            return response.json()
        except Exception as e:
            print(self.ERROR_MSG, e)
            return None


### 1. Get Access Token (Client Credentials Grant)

Retrieves an access token using the client credentials OAuth2 grant type.


In [ ]:
# Keycloak Client Init

keycloak_client = Simple_Keycloak_Client()

# Getting access token
token_response = keycloak_client.get_access_token()
token_response


### 2. Introspect Token

Validates a token and retrieves its metadata.


In [ ]:
# Introspecting the access token

if keycloak_client.access_token:
    introspection_result = keycloak_client.introspect_token(keycloak_client.access_token)
    introspection_result


### 3. Get User Info

Retrieves information about the authenticated user from the UserInfo endpoint.


In [ ]:
# Getting user information

if keycloak_client.access_token:
    user_info = keycloak_client.get_user_info()
    user_info


### 4. Get OpenID Connect Discovery Document

Retrieves the well-known OpenID Connect configuration for the realm.


In [ ]:
# Getting OpenID Connect discovery document

well_known_config = keycloak_client.get_well_known_config()
well_known_config


## MinIO


<ol>
<li> Python-based: <b>Available</b> (via official minio-py library)
<li> Open-source: <b>Free</b> (AGPL v3 License)

<li> Maximum Rate Limit Restriction:
<ul>
<li> Self-hosted: <b>No rate limits</b> (depends on infrastructure).
<li> Supports high-throughput object storage operations.
</ul>

<li> <b>Supports Core Architectural Choice</b> (S3-compatible API, async support via aiobotocore)
<li> Service is confirmed to <b>still be active & accessible</b>
</ol>

### Expected Basic Operations

1. [Create Bucket](https://min.io/docs/minio/linux/developers/python/API.html#make_bucket).
2. [Upload Object (Document)](https://min.io/docs/minio/linux/developers/python/API.html#put_object).
3. [Download Object](https://min.io/docs/minio/linux/developers/python/API.html#get_object).
4. [List Objects in Bucket](https://min.io/docs/minio/linux/developers/python/API.html#list_objects).
5. [Delete Object](https://min.io/docs/minio/linux/developers/python/API.html#remove_object).
6. [Generate Presigned URL](https://min.io/docs/minio/linux/developers/python/API.html#presigned_get_object).

**<u>Notes:</u>**
1. MinIO is S3-compatible, meaning existing S3 code/tools work with minimal changes.
2. Supports server-side encryption for secure document storage.
3. Can be deployed in distributed mode for high availability.


### Setup

MinIO provides an S3-compatible object storage API. We'll create a client to handle document upload operations for identity verification.


In [ ]:
# Class Definition for MinIO operations

class Simple_MinIO_Client():
    
    def __init__(
        self,
        endpoint: str = None,
        access_key: str = None,
        secret_key: str = None,
        secure: bool = False
    ):
        self.endpoint = endpoint or os.getenv("MINIO_ENDPOINT", "localhost:9000")
        self.access_key = access_key or os.getenv("MINIO_ACCESS_KEY", "minioadmin")
        self.secret_key = secret_key or os.getenv("MINIO_SECRET_KEY", "minioadmin")
        self.secure = secure
        self.SUCCESS_MSG = "[INFO]: Operation performed successfully!"
        self.ERROR_MSG = "[WARNING]:"
        
        self.client = Minio(
            self.endpoint,
            access_key=self.access_key,
            secret_key=self.secret_key,
            secure=self.secure
        )
    
    def create_bucket(self, bucket_name: str):
        """Create a new bucket if it doesn't exist"""
        try:
            if not self.client.bucket_exists(bucket_name):
                self.client.make_bucket(bucket_name)
                print(self.SUCCESS_MSG, f"Bucket '{bucket_name}' created.")
            else:
                print(f"[INFO]: Bucket '{bucket_name}' already exists.")
            return True
        except S3Error as e:
            print(self.ERROR_MSG, e)
            return False
    
    def upload_document(self, bucket_name: str, object_name: str, data: bytes, content_type: str = "application/octet-stream"):
        """Upload a document to the specified bucket"""
        try:
            data_stream = io.BytesIO(data)
            result = self.client.put_object(
                bucket_name,
                object_name,
                data_stream,
                length=len(data),
                content_type=content_type
            )
            print(self.SUCCESS_MSG)
            return result
        except S3Error as e:
            print(self.ERROR_MSG, e)
            return None
    
    def download_document(self, bucket_name: str, object_name: str):
        """Download a document from the specified bucket"""
        try:
            response = self.client.get_object(bucket_name, object_name)
            data = response.read()
            response.close()
            response.release_conn()
            print(self.SUCCESS_MSG)
            return data
        except S3Error as e:
            print(self.ERROR_MSG, e)
            return None
    
    def list_documents(self, bucket_name: str, prefix: str = ""):
        """List all documents in a bucket"""
        try:
            objects = self.client.list_objects(bucket_name, prefix=prefix)
            document_list = [obj.object_name for obj in objects]
            print(self.SUCCESS_MSG)
            return document_list
        except S3Error as e:
            print(self.ERROR_MSG, e)
            return []
    
    def delete_document(self, bucket_name: str, object_name: str):
        """Delete a document from the bucket"""
        try:
            self.client.remove_object(bucket_name, object_name)
            print(self.SUCCESS_MSG)
            return True
        except S3Error as e:
            print(self.ERROR_MSG, e)
            return False
    
    def get_presigned_url(self, bucket_name: str, object_name: str, expires_hours: int = 1):
        """Generate a presigned URL for temporary access"""
        from datetime import timedelta
        try:
            url = self.client.presigned_get_object(
                bucket_name,
                object_name,
                expires=timedelta(hours=expires_hours)
            )
            print(self.SUCCESS_MSG)
            return url
        except S3Error as e:
            print(self.ERROR_MSG, e)
            return None


### 1. Create Bucket

Creates a bucket for storing identity verification documents.


In [ ]:
# MinIO Client Init

minio_client = Simple_MinIO_Client()

# Creating bucket for identity documents
bucket_name = "identity-documents"
minio_client.create_bucket(bucket_name)


### 2. Upload Document

Uploads an identity verification document to the bucket.


In [ ]:
# Uploading a sample document

sample_document = b"Sample identity document content for testing purposes."
object_name = "user_123/passport_scan.txt"

minio_client.upload_document(
    bucket_name=bucket_name,
    object_name=object_name,
    data=sample_document,
    content_type="text/plain"
)


### 3. List Documents

Lists all documents in the identity verification bucket.


In [ ]:
# Listing documents in the bucket

documents = minio_client.list_documents(bucket_name)
print(f"Documents in '{bucket_name}':", documents)


### 4. Download Document

Downloads a previously uploaded identity document.


In [ ]:
# Downloading the document

downloaded_data = minio_client.download_document(bucket_name, object_name)
if downloaded_data:
    print(f"Downloaded content: {downloaded_data.decode('utf-8')}")


### 5. Generate Presigned URL

Generates a temporary URL for secure document access.


In [ ]:
# Generating presigned URL for temporary access

presigned_url = minio_client.get_presigned_url(bucket_name, object_name, expires_hours=24)
print(f"Presigned URL (valid for 24 hours):\n{presigned_url}")


### 6. Delete Document

Removes a document from the bucket.


In [ ]:
# Deleting the document

minio_client.delete_document(bucket_name, object_name)
